# [OpenCV-Python Tutorial] Feature Matching

In this notebook, we will see how to extract SIFT(Scale-Invariant Feature Transform) and match SIFT features of two images with OpenCV-Python.

---

In [ ]:
import numpy as np
import cv2 # OpenCV-Python
%matplotlib inline
import matplotlib.pyplot as plt
import time
from skimage import io

plt.style.use('default')
print("OpenCV-Python Version %s" % cv2.__version__)
home_url = r'https://raw.githubusercontent.com/InovaDx/public/master/OpenCV/'

### Loading images
Loading images from a local disk unit is done with the following code:
```python
input_image = cv2.imread('filepath/image.png', cv2.IMREAD_COLOR)
```
When running this notebook from Google Colaboratory, images are not local, and the way to get to open them changes. A different library must be used (`skimage`) in order to fetch the hyperlink to the image and open it as an array of pixels.
One of the main differences between both methods is the fact that `OpenCV`, unless specified, always opens images as RGB arrays with 3 layers, while `skimage` only creates the required layers.
That results in some inconsistencies in case a grayscale image is loaded, since only 1 layer is loaded by `skimage`, but `OpenCV` loads all 3 RGB.

This notebook assumes it is used from Google Colab and will always use the `skimage` option, even though is less common than using `OpenCV` in real life.

In [ ]:
# Load an image
filename='images/beaver.png'
beaver = io.imread(home_url+filename)
plt.imshow(beaver, cmap='gray')

In [ ]:
# Members of cv2.xfeatures2d
dir(cv2.xfeatures2d)

# Extract SIFT features from an image

## 1. SIFT detector

In [ ]:
# Convert image color(BGR->Grayscale)
gray = beaver
# You can convert the image when calling cv2.imread()
# gray = cv2.imread('images/beaver.png', cv2.IMREAD_GRAYSCALE)

print(str(beaver.shape) + " => " + str(beaver.shape))
plt.imshow(gray, cmap='gray')

In [ ]:
# SIFT feature detector/descriptor
sift = cv2.xfeatures2d.SIFT_create()

In [ ]:
# SIFT feature detection
start_time = time.time()
# kp = sift.detect(gray, None) # 2nd pos argument is a mask indicating a part of image to be searched in
kp = sift.detect(beaver, None) # 2nd pos argument is a mask indicating a part of image to be searched in
print('Elapsed time: %.6fs' % (time.time() - start_time))

In [ ]:
# Display the SIFT features
beaver_sift = cv2.drawKeypoints(beaver, kp, None)
plt.imshow(beaver_sift)

In [ ]:
# Display the rich SIFT features
beaver_sift2 = cv2.drawKeypoints(beaver, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(beaver_sift2)

In [ ]:
# Inspect the keypoints
print (type(kp))
print (len(kp))

In [ ]:
print (type(kp[0]))
print (dir(kp[0]))

In [ ]:
# A keypoint's property
# kp is sorted by scale of the keypoints
print (kp[-1].angle) # Orientation
print (kp[-1].class_id)
print (kp[-1].octave)
print (kp[-1].pt) # Position
print (kp[-1].response)
print (kp[-1].size) # Scale

## 2. Extract SIFT descriptor

In [ ]:
# Extract SIFT feature from the (gray) image and detected keypoints
start_time = time.time()
kp, des = sift.compute(gray, kp)
print('Elapsed time: %.6fs' % (time.time() - start_time))

# SIFT keypoints and descriptors at the same time
# start_time = time.time()
# kp, des = sift.detectAndCompute(gray, None)
# print('Elapsed time: %.6fs' % (time.time() - start_time))

In [ ]:
# Inspect the descriptors
print (type(des))
print (des.shape)
print (des.dtype)

In [ ]:
print (len(des[0, :]))
print (des[0, :])

# Feature Matching

## 1. SIFT Feature Matching

In [ ]:
# Open and show images
# img1 = cv2.imread('images/box.png')
# img2 = cv2.imread('images/box_in_scene.png')

img1 = io.imread(home_url+'images/box.png')
img2 = io.imread(home_url+'images/box_in_scene.png')

plt.subplot(1, 2, 1)
plt.imshow(img1, cmap='gray')
plt.subplot(1, 2, 2)
plt.imshow(img2, cmap='gray')

In [ ]:
# SIFT feature extracting
sift = cv2.xfeatures2d.SIFT_create()
# gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
# gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
gray1 = img1
gray2 = img2

start_time = time.time()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)
print('Elapsed time: %.6fs' % (time.time() - start_time))

print('Image 1 - %d feature detected' % des1.shape[0])
print('Image 2 - %d feature detected' % des2.shape[0])

In [ ]:
# BFMatcher(Brute Force Matcher) with defalut setting
bf = cv2.BFMatcher(cv2.NORM_L2)
matches = bf.knnMatch(des1, des2, k=2)
print('%d matches' % len(matches))

In [ ]:
# Inspect matcher results
print (type(matches))
print (len(matches))
print (type(matches[0]))
print (len(matches[0]))

In [ ]:
print (type(matches[0][0]))
print (dir(matches[0][0]))

In [ ]:
print (matches[0][0].distance)
print (matches[0][0].queryIdx)
print (matches[0][0].trainIdx)
print (matches[0][0].imgIdx)
print (matches[0][1].distance)
print (matches[0][1].queryIdx)
print (matches[0][1].trainIdx)
print (matches[0][1].imgIdx)

In [ ]:
# Apply ratio test as in David Rowe's paper
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)
print('%d matches' % len(good_matches))

In [ ]:
# Display matches
img3 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None)
plt.imshow(cv2.cvtColor(img3, cv2.COLOR_BGR2RGB))

## 2. SIFT Mathing with Hellinger Distance

In [ ]:
# L1 normalization
des1 = des1 / np.repeat(np.sum(des1, axis = 1).reshape(des1.shape[0], 1), des1.shape[1], axis=1)
des2 = des2 / np.repeat(np.sum(des2, axis = 1).reshape(des2.shape[0], 1), des2.shape[1], axis=1)

# Calculate Hellinger distance for every feature pair
dist_mat = np.sqrt(1.0 - np.dot(np.sqrt(des1), np.sqrt(des2).transpose()))

# Match with ratio test
min_arg = np.argsort(dist_mat, axis=1)
good_matches = []
for i in range(dist_mat.shape[0]):
    m, n = min_arg[i][0:2]
    if dist_mat[i][m] < dist_mat[i][n] * 0.75:
        dmatch = cv2.DMatch(i, m, 0, dist_mat[i][m]) # _queryIdx, _trainIdx, _imgIdx, _distance
        good_matches.append(dmatch)

print('%d matches' % len(good_matches))

In [ ]:
img4 = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None)
plt.imshow(cv2.cvtColor(img4, cv2.COLOR_BGR2RGB))